In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
# THIS FILE HAS BEEN MODIFIED

import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [19]:
#bring the model back in
dataset_name = 'imdb'
saved_model_path = './{}_bert'.format(dataset_name.replace('/', '_'))
reloaded_model = tf.saved_model.load(saved_model_path)

In [20]:
with open("complete_data.csv", "r", encoding="utf-8") as f:
    df = pd.read_csv(f)

df_sentence = pd.DataFrame(df.columns[[1]])
#list_sentence = df_sentence.values.tolist()
list_sentence = ["not good", "The Rlng-tum Phl, October 19, 1989  Abortion: The Final Solution?"]

results = tf.sigmoid(reloaded_model(tf.constant(list_sentence)))
print(results)

tf.Tensor(
[[0.03447151]
 [0.47144744]], shape=(2, 1), dtype=float32)


In [ ]:
df_sentence.head()

In [ ]:
df_final = pd.DataFrame({
    'id': df.columns[[0]],
    'label': results,
    'sentence': df.columns[[1]],
    'years': df.columns[[2]],
    'keyword': df.columns[[3]]
})
df_final.head()

In [ ]:
df_final.to_csv("final_data.csv", index=False, header=False)